# Alfred Annontation Task Demo

## 1. Load Dataset!

In [45]:
from alfred.data.wrench import WrenchBenchmarkDataset

youtube_train = WrenchBenchmarkDataset(
                                dataset_name='youtube',
                                split='test',
                                local_path="/home/peilin/Dataset/wrench/"
                            )

## 2. Write Templates!

In [46]:
from alfred.template import StringTemplate


video_template = StringTemplate(
    template = """Does the following comment reference the speaker’s channel or video?\n\n[text]""",
    answer_choices = "yes ||| no",
    label_maps = {'yes': 2},
)

prompts = [video_template.apply(instance) for instance in youtube_train]

## 3. Run an Alfred Client

In [47]:
from alfred.client import Client


T0pp = Client(end_point="pyu12@ssh.ccv.brown.edu:10719", ssh_tunnel=True, ssh_node="gpu1404")



Password: ········


Duo two-factor login for pyu12

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-0672
 2. Duo Push to XXX-XXX-6788
 3. Phone call to XXX-XXX-0672
 4. Phone call to XXX-XXX-6788
 5. SMS passcodes to XXX-XXX-0672
 6. SMS passcodes to XXX-XXX-6788

Passcode or option (1-6): 1




In [48]:
label2id = {'SPAM': 1, 'HAM': 0}

ytb_prompts = """Does the following comment reference the speaker’s channel or video?\n\n[text]
||| Does the following comment ask you to subscribe to a channel?\n\n[text]
||| Does the following comment have a URL?\n\n[text]
||| Does the following comment ask the reader to do something?\n\n[text]
||| Does the following comment talk about a song?\n\n[text]
||| Does the following comment contain the words "check out"? \n\n[text]
||| Is the following comment fewer than 5 words?\n\n[text]
||| Does the following comment mention a person’s name?\n\n[text]
||| Does the following comment express a very strong sentiment?\n\n[text]
||| Does the following comment express a subjective opinion?\n\n[text]""".split('|||')

prompts_labels = [label2id[label] for label in "SPAM SPAM SPAM SPAM HAM SPAM HAM HAM HAM HAM".split()]


templates = [
    StringTemplate(
        template = prompt,
        answer_choices = "yes ||| no",
        label_maps = {'yes': prompts_labels[prompt_id], 'no': abs(prompts_labels[prompt_id]-1)},
        )
    for prompt_id, prompt in enumerate(ytb_prompts)
]

prompts = [
    [
        template.apply(example) for
        example in youtube_train
    ] for template in templates
]

In [49]:
import numpy as np
from tqdm.auto import tqdm

votes = np.zeros([len(youtube_train), len(ytb_prompts)])


for template_id, template in enumerate(tqdm(templates)):
    prompts = [template.apply(example) for example in youtube_train]
    votes[:, template_id] = template.vote(T0pp(prompts))

  0%|          | 0/10 [00:00<?, ?it/s]

In [50]:
import scipy

mv_labels = scipy.stats.mode(votes, axis=1).mode.flatten()

In [55]:
np.mean(mv_labels==youtube_train.labels)

0.508